[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Vampire-Chan/SDUI/blob/main/ComfyUI.ipynb)


In [1]:
#@title **ComfyUI Environment Auto-Setup with Update Support**
import os
from IPython.display import clear_output

# ✅ User-configurable options
USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}

# ✅ Define workspace path
if USE_GOOGLE_DRIVE:
    print("🔗 Mounting Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
else:
    WORKSPACE = "./ComfyUI"

COMFY_REPO = "https://github.com/comfyanonymous/ComfyUI"
MANAGER_REPO = "https://github.com/ltdrdata/ComfyUI-Manager.git"
CUSTOM_NODES_DIR = os.path.join(WORKSPACE, "custom_nodes", "ComfyUI-Manager")

# ✅ Clone or update ComfyUI
if not os.path.exists(WORKSPACE):
    print("📂 Cloning ComfyUI for the first time...")
    os.system(f"git clone {COMFY_REPO} {WORKSPACE}")
else:
    if UPDATE_COMFY_UI:
        print("🔄 Updating ComfyUI...")
        os.chdir(WORKSPACE)
        os.system("git reset --hard")
        os.system("git pull origin master")
    else:
        print("⏭️ Skipping ComfyUI update.")

# ✅ Clone or update ComfyUI-Manager
if not os.path.exists(CUSTOM_NODES_DIR):
    print("📂 Cloning ComfyUI-Manager for the first time...")
    os.system(f"git clone {MANAGER_REPO} {CUSTOM_NODES_DIR}")
else:
    if UPDATE_COMFY_UI:
        print("🔄 Updating ComfyUI-Manager...")
        os.chdir(CUSTOM_NODES_DIR)
        os.system("git reset --hard")
        os.system("git pull origin main")
    else:
        print("⏭️ Skipping ComfyUI-Manager update.")

# ✅ Install dependencies
os.chdir(WORKSPACE)
print("📦 Installing dependencies...")
os.system(
    "pip install xformers!=0.0.18 -r requirements.txt "
    "--extra-index-url https://download.pytorch.org/whl/cu121 "
    "--extra-index-url https://download.pytorch.org/whl/cu118 "
    "--extra-index-url https://download.pytorch.org/whl/cu117"
)

print("✅ Environment setup and update complete.")
clear_output()


In [ ]:
import os
import requests
import shutil
import urllib.parse
import time
import sys

# 🔄 Unified dropdown for selecting storage location
MODEL_STORAGE_LOCATION = "Google Drive (Persistent)"  #@param ["Google Drive (Persistent)", "Colab Filesystem (Notebook)", "Temporary SDUI Folder (Volatile)"]
MODEL_URL = "https://tensor-models.7022ae40757f8d53295a57619de9b364.r2.cloudflarestorage.com/files/762555264535746522/f7278d22-91b0-44ea-ad8c-9c5bc002e315.safetensors"  #@param {type:"string"}
API_KEY = ""  #@param {type:"string"}
MODEL_TYPE = "checkpoints"  #@param ["checkpoints", "clip_vision", "vae", "loras", "controlnet", "style_models", "upscale_models", "diffusion_models", "hypernetworks", "gligen", "custom_nodes"]

# 🔧 Determine ComfyUI base path from dropdown
def setup_environment():
    if MODEL_STORAGE_LOCATION == "Google Drive (Persistent)":
        print("🔗 Mounting Google Drive...")
        from google.colab import drive
        drive.mount('/content/drive')
        base_dir = '/content/drive/MyDrive/ComfyUI'
    elif MODEL_STORAGE_LOCATION == "Colab Filesystem (Notebook)":
        base_dir = './ComfyUI'
    else:
        base_dir = './content/sdui'
    return base_dir

# 🌍 Setup environment
BASE_DIR = setup_environment()

# 📁 Model destination folders from ComfyUI config structure
MODEL_DIRS = {
    'checkpoints': 'models/checkpoints',
    'clip_vision': 'models/clip_vision',
    'vae': 'models/vae',
    'loras': 'models/loras',
    'controlnet': 'models/controlnet',
    'style_models': 'models/style_models',
    'upscale_models': 'models/upscale_models',
    'diffusion_models': 'models/diffusion_models',
    'hypernetworks': 'models/hypernetworks',
    'gligen': 'models/gligen',
    'custom_nodes': 'custom_nodes',
    'clip': 'models/clip',
    'configs': 'models/configs',
}

# 🔁 Build full paths
MODEL_DIRS = {key: os.path.join(BASE_DIR, path) for key, path in MODEL_DIRS.items()}

# 🏗️ Make sure the model folders exist
for path in MODEL_DIRS.values():
    os.makedirs(path, exist_ok=True)

def format_progress_bar(percent, bar_length=30):
    filled_length = int(bar_length * percent // 100)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    return f"[{bar}]"

# ⬇️ Download logic with progress
def download_file(url, save_path, api_key=None):
    try:
        headers = {}
        if api_key:
            headers['Authorization'] = f'Bearer {api_key}'

        response = requests.get(url, stream=True, allow_redirects=True, headers=headers)

        if response.status_code == 403 and not api_key:
            print("⚠️ Access denied. Retrying with API key...")
            return download_file(url, save_path, api_key=API_KEY)

        response.raise_for_status()

        filename = os.path.basename(urllib.parse.urlparse(url).path)
        if 'content-disposition' in response.headers:
            import cgi
            _, params = cgi.parse_header(response.headers['content-disposition'])
            filename = params.get('filename', filename)

        save_path = os.path.join(save_path, filename)

        total_size = int(response.headers.get('Content-Length', 0))
        downloaded = 0
        start_time = time.time()

        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    downloaded += len(chunk)

                    percent = (downloaded / total_size) * 100
                    elapsed_time = time.time() - start_time
                    speed = downloaded / (1024 * 1024 * elapsed_time)  # MB/s
                    time_left = (total_size - downloaded) / (speed * 1024 * 1024) if speed > 0 else 0
                    progress_bar = format_progress_bar(percent)

                    sys.stdout.write(
                        f"\r📥 Downloading {filename} {progress_bar} {percent:.2f}% "
                        f"{downloaded / (1024 * 1024):.2f}MB / {total_size / (1024 * 1024):.2f}MB | "
                        f"{speed:.2f} MB/s | ETA: {time_left:.2f}s"
                    )
                    sys.stdout.flush()

        print(f"\n✅ Downloaded: {filename} to {save_path}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error downloading {url}: {e}")

# 🚀 Start the download process
if MODEL_URL:
    print(f"⏳ Starting download for {MODEL_URL} to ComfyUI/{MODEL_TYPE}...")
    save_path = MODEL_DIRS.get(MODEL_TYPE, os.path.join(BASE_DIR, 'models'))
    download_file(MODEL_URL, save_path, api_key=API_KEY)
else:
    print("❌ No URL provided for download.")


⏳ Starting download for https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/diffusion_models/wan2.1_i2v_720p_14B_fp16.safetensors to the 'diffusion_models' directory...

In [2]:
#@title 🔧 ComfyUI Launcher (with VRAM Flags, Logs, Drive Options)
#@markdown ### Setup Options
storage_location = "Google Drive"  #@param ["Google Drive", "Colab Runtime", "SDUI Temp Folder"]
show_logs = True  #@param {type:"boolean"}
custom_flags = "--highvram --dont-print-server"  #@param {type:"string"}
required_modules = ['torch', 'flask', 'flask_cors', 'requests', 'torchsde', 'spandrel', 'kornia', 'piexif', 'segment_anything']  #@param

import os, subprocess, threading, time, socket
from google.colab import drive

# ✅ Determine folder path
if storage_location == "Google Drive":
    print("🔗 Mounting Google Drive...")
    drive.mount('/content/drive')
    folder_path = "/content/drive/MyDrive/ComfyUI"
elif storage_location == "SDUI Temp Folder":
    folder_path = "/content/sdui/ComfyUI"
else:
    folder_path = "/content/ComfyUI"

os.makedirs(folder_path, exist_ok=True)
print(f"📁 Using storage at: {folder_path}")

# ✅ Check CUDA
def check_cuda():
    import torch
    print("🚀 CUDA Available!" if torch.cuda.is_available() else "⚠️ No GPU found, running on CPU.")
check_cuda()

# ✅ Install missing modules
def install_module(module):
    try:
        __import__(module)
    except ModuleNotFoundError:
        print(f"📦 Installing missing module: {module}")
        os.system(f"pip install {module}")
        print(f"✅ {module} installed.")

for mod in required_modules:
    install_module(mod)

# ✅ Download and install Cloudflared
cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"
cloudflared_deb = os.path.join(folder_path, "cloudflared.deb")

if not os.path.exists("/usr/bin/cloudflared"):
    print("📥 Downloading Cloudflared...")
    os.system(f"wget -q --show-progress {cloudflared_url} -O {cloudflared_deb}")
    print("🛠️ Installing Cloudflared...")
    os.system(f"dpkg -i {cloudflared_deb}")

# ✅ Tunnel Launch Function
def launch_cloudflared(port):
    while True:
        time.sleep(0.5)
        try:
            with socket.create_connection(("127.0.0.1", port), timeout=1):
                break
        except:
            pass

    print("\n🌐 ComfyUI is ready. Launching Cloudflared tunnel...\n")
    process = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    for line in process.stderr:
        decoded = line.decode()
        if "trycloudflare.com" in decoded:
            print("🔗 ComfyUI Public URL:", decoded[decoded.find("http"):], end='')

threading.Thread(target=launch_cloudflared, daemon=True, args=(8188,)).start()

# ✅ Start ComfyUI
print(f"🚀 Starting ComfyUI with flags: {custom_flags}")
main_path = os.path.join(folder_path, "main.py")

if show_logs:
    os.system(f"python {main_path} {custom_flags}")
else:
    os.system(f"python {main_path} {custom_flags} > /dev/null 2>&1 &")
    print("✅ ComfyUI is running silently. Enable 'Show Logs' to see output.")


🔗 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Using storage at: /content/drive/MyDrive/ComfyUI
🚀 CUDA Available!


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
#@title **📦 Compress & Clean Output Folder**
#@markdown Use this tool to compress your output folder and clean it after.
#@markdown Choose file format, compression level, and destination.
!pip install py7zr
!pip install rarfile
import os
import shutil
import zipfile
import sys
import subprocess
import py7zr
import rarfile
from tqdm import tqdm
from google.colab import drive

# ✅ **Mount Google Drive**
mount_gdrive = True  #@param {type:"boolean"}
if mount_gdrive:
    drive.mount('/content/drive')

# ✅ **User Options**
output_folder = "/content/drive/MyDrive/ComfyUI/output"  #@param {type: "string"}
archive_name = "output"  #@param {type: "string"}
archive_destination = "/content/drive/MyDrive/"  #@param {type: "string"}

file_type = "zip"  #@param ["zip", "7z", "rar"]
compression_level = "normal"  #@param ["store", "fast", "normal", "high", "maximum"]

# 🔥 **Compression Level Mapping**
compression_map = {
    'store': 0, 'fast': 1, 'normal': 5, 'high': 7, 'maximum': 9
}

def compress_and_clean_output(output_folder, archive_name, archive_destination, file_type, compression_level):
    """Compresses and cleans the output folder."""

    if not os.path.exists(output_folder):
        print(f"❌ Error: The folder '{output_folder}' does not exist.")
        return

    if not os.path.exists(archive_destination):
        print(f"❌ Error: The destination folder '{archive_destination}' does not exist.")
        return

    total_original_size = sum(
        os.path.getsize(os.path.join(root, file))
        for root, _, files in os.walk(output_folder)
        for file in files
    )

    extension = {'zip': '.zip', '7z': '.7z', 'rar': '.rar'}[file_type]
    archive_path = os.path.join(archive_destination, f"{archive_name}{extension}")
    compression_value = compression_map[compression_level]

    # ✅ **ZIP Compression**
    if file_type == 'zip':
        compression = zipfile.ZIP_STORED if compression_level == 'store' else zipfile.ZIP_DEFLATED
        with zipfile.ZipFile(archive_path, 'w', compression) as zipf:
            for root, _, files in os.walk(output_folder):
                for file in tqdm(files, desc="📦 Compressing as .zip", unit="file"):
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_folder))

    # ✅ **7Z Compression**
    elif file_type == '7z':
        with py7zr.SevenZipFile(archive_path, 'w', filters=[{'id': py7zr.FILTER_LZMA2, 'preset': compression_value}]) as archive:
            for root, _, files in os.walk(output_folder):
                for file in tqdm(files, desc="📦 Compressing as .7z", unit="file"):
                    archive.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_folder))

    # ✅ **RAR Compression**
    elif file_type == 'rar':
        try:
            print("📦 Compressing as .rar (This may take some time, progress bar not supported)")
            subprocess.run(['rar', 'a', '-m' + str(compression_value), archive_path, output_folder], check=True)
        except subprocess.CalledProcessError:
            print("❌ Failed to create .rar archive. Install 'rar' using 'apt-get install rar'.")

    total_compressed_size = os.path.getsize(archive_path)
    compression_percentage = (1 - (total_compressed_size / total_original_size)) * 100 if total_original_size > 0 else 0

    print(f"📦 Original size: {total_original_size / 1024**2:.2f} MB")
    print(f"📦 Compressed size: {total_compressed_size / 1024**2:.2f} MB")
    print(f"📦 Compression percentage: {compression_percentage:.2f}%")

    # ✅ **Delete original files**
    for filename in os.listdir(output_folder):
        file_path = os.path.join(output_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'❌ Failed to delete {file_path}. Reason: {e}')

    print(f"✅ Successfully compressed and cleaned {output_folder}")

# 🔥 **Run the function**
compress_and_clean_output(output_folder, archive_name, archive_destination, file_type, compression_level)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


📦 Compressing as .zip: 100%|██████████| 70/70 [00:18<00:00,  3.85file/s]


📦 Original size: 336.22 MB
📦 Compressed size: 335.62 MB
📦 Compression percentage: 0.18%
✅ Successfully compressed and cleaned /content/drive/MyDrive/ComfyUI/output
